In [1]:
# Interactive Figure
%matplotlib ipympl 
# Non interactive
#%matplotlib inline 

RuntimeError: 'ipympl' is not a recognised GUI loop or backend name

In [2]:
import time
import numpy as np
from matplotlib import pyplot as plt
from qualang_tools.units import unit
u = unit(coerce_to_integer=True)
from qm.qua import *
from qm import QuantumMachinesManager
from qm import SimulationConfig
from qualang_tools.results import fetching_tool
from qualang_tools.loops import from_array
import config_00 as config
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'qualang_tools'

In [3]:
def addjob(qmprog, qm):
    # Add a QUA program to the OPX queue, which compiles it and executes it
    job = qm.queue.add(qmprog)
    # Wait for job to be loaded
    while job.status=="loading":
        print("Job is loading...")
        time.sleep(0.1)
    # Wait until job is running
    time.sleep(0.1)
    while job.status=="pending":
        q = job.position_in_queue()
        if q>0:
            print("Position in queue",q,end='\r')
        time.sleep(0.1)
    job=job.wait_for_execution()
    print("\nJob is running")
    return job

In [4]:
import threading
from IPython.display import display
import ipywidgets as widgets

class JobAborter:
    def __init__(self,job,timelimit=10):
        self.job=job
        self.keeprunning=True
        self.progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)
        self.button = widgets.Button(description='Abort',)
        self.start_time = time.time()
        self.button.on_click(self.stop)
        self.timelimit = timelimit
        self.thread = threading.Thread(target=self.update,)
        self.thread.start()
    def stop(self,b):
        self.keeprunning = False
    def update(self):
        dt = 0
        while dt<1 and self.keeprunning and job.status=='running':
            dt = (time.time()-self.start_time)/self.timelimit
            self.progress.value = dt
            time.sleep(0.1)
        self.job.halt()
        print('Job finished')
    def show(self):
        display(widgets.HBox([self.button,self.progress,]))

In [5]:
# Connect to the cluster (run only once)
import QM_cluster
qmm = QuantumMachinesManager(host=QM_cluster.QM_Router_IP, cluster_name=QM_cluster.cluster_name)

2025-05-21 17:20:37,657 - qm - INFO     - Performing health check
2025-05-21 17:20:37,665 - qm - INFO     - Health check passed


# Connexion à la Quantum Machine

In [6]:
# Get the QM reference (rerun every time the config is changed)
qm_list =  qmm.list_open_qms()
qm = qmm.get_qm(qm_list[0])
print(f"Connected to {qm.id}")

Connected to qm-1690922101142


# Envoyer des pulses et les observer

In [7]:
with program() as prog:
    with infinite_loop_():
        play('trigger','trigger')
        # Pulse sequence
        play('pulse','qubit',duration=400*u.ns)
        wait(200*u.ns)
        play('pulse','qubit',duration=400*u.ns)
        wait(10*u.us)

In [9]:
# Run the code 
job = addjob(prog, qm)
job_aborter = JobAborter(job)
job_aborter.show()

2025-05-21 17:20:54,075 - qm - INFO     - Sending program to QOP for compilation

Job is running


Job finished


In [ ]:
job.halt()

# Exercices

## Exercice 1 : Variation du délai d’impulsion dans une boucle
- Modifier le délai entre deux impulsions carrées dans une boucle.
- Vérifier que vous obtenez le résultat attendu sur l’analyseur de spectre.

## Exercice 2 : Variation de l’amplitude d’impulsion dans une boucle
- Modifier l’amplitude de la seconde impulsion dans une boucle.
- Vérifier que vous obtenez le résultat attendu sur l’analyseur de spectre.

## Exercice 3 : Variation de la durée d’impulsion
- Modifier la durée de la seconde impulsion dans une boucle.
- Vérifier que vous obtenez le résultat attendu sur l’analyseur de spectre.
